In [1]:
import json
from paperqa import Settings, ask
import os
from paperqa import Settings, Docs
from paperqa.agents.main import agent_query
from paperqa.agents.search import get_directory_index

from aviary.env import TaskDataset
from ldp.agent import SimpleAgent
from ldp.alg.callbacks import MeanMetricsCallback
from ldp.alg.runners import Evaluator, EvaluatorConfig
import sys

In [2]:
sys.path.append("src")

In [3]:
from MC_Grader import MultipleChoiceGrader, process_results
from config import OPENAI_API_KEY
from openai import OpenAI

## Load the Question 

In [48]:
with open("/Users/apple/Documents/GitLab_Projects/master_project/xx823/Reproduction/formatted_questions_test/questions.json", "r") as f:
    questions = json.load(f)

In [49]:
print(questions)

[{'question': 'Approximately what percentage of topologically associated domains in the GM12878 blood cell line does DiffDomain classify as reorganized in the K562 cell line?', 'choices': ['(A) 41%', '(B) 51%', '(C) 31%', '(D) Insufficient information to answer the question', '(E) 11%', '(F) 21%'], 'correct_answer': 'C', 'unsure_option': 'D', 'sources': ['https://doi.org/10.1038/s41467-024-44782-6'], 'ideal': '31%', 'distractors': ['21%', '11%', '41%', '51%']}, {'question': 'At least how long do SynNotch-MCF10DCIS cells express BFP after contact with GFP+BMSC3 cells?', 'choices': ['(A) 0 h', '(B) 6 h', '(C) 24', '(D) 12 h', '(E) 48 h', '(F) 96 h', '(G) Insufficient information to answer the question', '(H) 72 h'], 'correct_answer': 'H', 'unsure_option': 'G', 'sources': ['https://doi.org/10.1073/pnas.2322688121'], 'ideal': '72 h', 'distractors': ['24', '48 h', '0 h', '12 h', '6 h', '96 h']}, {'question': 'DK015 and DK038 strains of Verticillium dahliae have in common approximately what 

In [50]:
def format_multiple_choice_question(question, choices):
    formatted = f"Question:Q: {question}; Options: "
    formatted += " ".join(choices)
    return formatted


In [51]:
formatted_q=[]

In [52]:
for q in questions:
    formatted_q.append(format_multiple_choice_question(q["question"], q["choices"]))

In [53]:
len(formatted_q)

49

In [32]:
print(formatted_q[:5])

['Question:Q: Approximately what percentage of topologically associated domains in the GM12878 blood cell line does DiffDomain classify as reorganized in the K562 cell line?; Options: (A) 41% (B) 51% (C) 31% (D) Insufficient information to answer the question (E) 11% (F) 21%', 'Question:Q: At least how long do SynNotch-MCF10DCIS cells express BFP after contact with GFP+BMSC3 cells?; Options: (A) 0 h (B) 6 h (C) 24 (D) 12 h (E) 48 h (F) 96 h (G) Insufficient information to answer the question (H) 72 h', 'Question:Q: DK015 and DK038 strains of Verticillium dahliae have in common approximately what percentage orthologous genes?; Options: (A) 97% (B) Insufficient information to answer the question (C) 95% (D) 98% (E) 96% (F) 94%', 'Question:Q: Expression of which of the following genes was significantly reduced in the cortical plate of olig2-/- mice compared to WT mice at E18.5?; Options: (A) none of the above (B) Insufficient information to answer the question (C) Aldh1l1 (D) Actin (E) MA

## Get the Paperqa Response

In [54]:
#build the index
# Create settings with explicit configuration
settings = Settings(
    paper_directory='/Users/apple/Documents/GitLab_Projects/master_project/xx823/papers',
    agent={"index": {
        "sync_with_paper_directory": True,
        "recurse_subdirectories": True
    }}
)

built_index = await get_directory_index(settings=settings)

# Print index information 
print(f"Using index: {settings.get_index_name()}")
index_files = await built_index.index_files
print(f"Number of indexed files: {len(index_files)}")
print("Indexed files:")
for file in index_files:
    print(f"- {file}")

Using index: pqa_index_73db35b7edb188828b5799b8ac0a3fac
Number of indexed files: 49
Indexed files:
- DiffDomain enables identification of structurally reorganized topologically associating domains.pdf
- Full-length tRNAs lacking a functional CCA tail are selectively sorted into the lumen of extracellular vesicles.pdf
- Type-I-interferon-responsive microglia shape cortical development and behavior.pdf
- liang-et-al-2024-a-conserved-molecular-logic-for-neurogenesis-to-gliogenesis-switch-in-the-cerebral-cortex.pdf
- goyette-et-al-2024-cancer-stromal-cell-interactions-in-breast-cancer-brain-metastases-induce-glycocalyx-mediated.pdf
- Light regulates widespread plant alternative polyadenylation 2 through the chloroplast.pdf
- High-speed imaging of giant unilamellar vesicle formation in cDICE.pdf
- elife-90425-v2.pdf
- sherman-et-al-2024-the-fatty-liver-disease-causing-protein-pnpla3-i148m-alters-lipid-droplet-golgi-dynamics.pdf
- Role of m6A RNA methylation in dosage compensation.pdf
- Func

In [62]:
def get_paperqa2_response(question_data):
    """
    This function should call PaperQA2 with the question and return the response.
    Adjust this function based on how you're currently calling PaperQA2.
    """
    # Load the settings
    settings = Settings(
    temperature=0.5,  # Keep deterministic
    paper_directory="/Users/apple/Documents/GitLab_Projects/master_project/xx823/papers"  # Directory containing the paper with DOI: 10.1101/2024.01.31.578101
)

    # Format the question as needed for PaperQA2
    #format_multiple_choice_question(question_data['question'], question_data['choices'])

    import nest_asyncio
    nest_asyncio.apply()
    # Call PaperQA2 with the question
    response_pqa = ask(question_data, settings=settings)
    answer_dict=response_pqa.dict()

    return answer_dict['session']['answer']


## Create the required data structure

In [56]:
grader_input_data = []

In [57]:
formatted_q[0]

'Question:Q: Approximately what percentage of topologically associated domains in the GM12878 blood cell line does DiffDomain classify as reorganized in the K562 cell line?; Options: (A) 41% (B) 51% (C) 31% (D) Insufficient information to answer the question (E) 11% (F) 21%'

In [ ]:
total_questions = len(formatted_q)
for i, q in enumerate(formatted_q[:10]): # test for 10 questions
    current_question_num = i + 1
    print(f"Processing question {current_question_num}/{total_questions}: {q[:50]}...")
    
    # Get PaperQA2's response for this question
    response = get_paperqa2_response(q)
    
    # Log completion status
    print(f"Completed question {current_question_num}/{total_questions}, {total_questions - current_question_num} remaining")
    
    # Add the question and response to the grader input
    grader_input_data.append({
        "question":questions[i]["question"],
        "choices": questions[i]["choices"],
        "response": response,
        "correct_answer": questions[i]["correct_answer"],
        "unsure_option": questions[i]["unsure_option"]
    })

print(f"All {total_questions} questions processed successfully")


2025-02-27 20:06:12,169 - LiteLLM - WARNING - Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30
2025-02-27 20:06:12,172 - LiteLLM - WARNING - Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30
2025-02-27 20:06:12,172 - LiteLLM - WARNING - Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30
2025-02-27 20:06:12,173 - LiteLLM - WARNING - Cannot add callback - would exceed MAX_CALLBACKS limit of 30. Current callbacks: 30


Processing question 1/49: Question:Q: Approximately what percentage of topol...


[20:06:13] Starting paper search for 'DiffDomain classify topologically associated domains GM12878 K562'.

2025-02-27 20:06:13,762 - paperqa.agents.tools - INFO - Starting paper search for 'DiffDomain classify topologically associated domains GM12878 K562'.


           paper_search for query 'DiffDomain classify topologically associated domains GM12878 K562' and offset 0 
           returned 8 papers.

...

2025-02-27 20:09:06,232 - paperqa.agents.tools - INFO - Status: Paper Count=7 | Relevant Papers=1 | Current Evidence=7 | Current Cost=$0.0627


[20:09:07] Completing 'Question:Q: For which of the following Trub1 substrates did positional occupancy increase   
           upon differentiation?; Options: (A) NKAIN1 (B) CCDC22 (C) HECTD1 (D) Insufficient information to answer 
           the question (E) FBXO5 (F) SCP2 (G) IDI1' as 'certain'.

2025-02-27 20:09:07,356 - paperqa.agents.tools - INFO - Completing 'Question:Q: For which of the following Trub1 substrates did positional occupancy increase upon differentiation?; Options: (A) NKAIN1 (B) CCDC22 (C) HECTD1 (D) Insufficient information to answer the question (E) FBXO5 (F) SCP2 (G) IDI1' as 'certain'.


           Answer: The Trub1 substrate IDI1 showed an increase in positional pseudouridylation (Ψ) occupancy upon  
           differentiation. Specifically, its mRNA expression significantly increased from 15.3 TPM (transcripts   
           per million) in the untreated sample to 58.9 TPM in the differentiated sample, and it was noted as a    
           Trub1 substrate (fanari2024probingenzymedependentpseudouridylation pages 4-5).                          
                                                                                                                   
           In contrast, NKAIN1 exhibited a decrease in positional occupancy, reducing from 25% in the untreated    
           sample to 12% after differentiation (fanari2024probingenzymedependentpseudouridylation pages 4-5). The  
           context does not provide specific information on positional occupancy changes for CCDC22, HECTD1, FBXO5,
           or SCP2, nor does it explicitly mention these as Trub1 substrates with increased occupancy.             
                                                                                                                   
           While some Trub1 substrates were classified as static, with no significant changes in occupancy under   
           differentiation or Pb2+ exposure, IDI1 stands out as a substrate with increased mRNA expression and     
           association with Trub1 (fanari2024probingenzymedependentpseudouridylation pages 7-8). Therefore, based  
           on the provided data, IDI1 is the only substrate among the options for which an increase in occupancy   
           upon differentiation is supported.

2025-02-27 20:09:07,451 - paperqa.agents.main.agent_callers - INFO - [bold blue]Answer: The Trub1 substrate IDI1 showed an increase in positional pseudouridylation (Ψ) occupancy upon differentiation. Specifically, its mRNA expression significantly increased from 15.3 TPM (transcripts per million) in the untreated sample to 58.9 TPM in the differentiated sample, and it was noted as a Trub1 substrate (fanari2024probingenzymedependentpseudouridylation pages 4-5). 

In contrast, NKAIN1 exhibited a decrease in positional occupancy, reducing from 25% in the untreated sample to 12% after differentiation (fanari2024probingenzymedependentpseudouridylation pages 4-5). The context does not provide specific information on positional occupancy changes for CCDC22, HECTD1, FBXO5, or SCP2, nor does it explicitly mention these as Trub1 substrates with increased occupancy. 

While some Trub1 substrates were classified as static, with no significant changes in occupancy under differentiation or Pb2+ expo

In [64]:
with open("formatted_questions/grader_input_data.json", "w") as f:
    json.dump(grader_input_data, f, indent=2)

In [4]:
#load the previous data if needed
with open("/Users/apple/Documents/GitLab_Projects/master_project/xx823/Reproduction/grader_input_data.json", 'r') as f:
        grader_input_data = json.load(f)
    

In [6]:
grader = MultipleChoiceGrader(
    # Add your OpenAI API key here if not set in environment
    openai_api_key=OPENAI_API_KEY,
    results_dir="grading_results_10MCQs_trial3"
)


In [7]:
# Use the fixed version instead of the original
metrics = process_results(grader, grader_input_data, save_prefix="paperqa2_evaluation")

2025-02-27 20:32:55,505 - MC_Grader - INFO - Processing question 1/10
2025-02-27 20:32:55,506 - MC_Grader - INFO - Grading question: Approximately what percentage of topologically ass...


HumanProxy (to GraderAgent):


    Question:
    Approximately what percentage of topologically associated domains in the GM12878 blood cell line does DiffDomain classify as reorganized in the K562 cell line?

    Options:
    (A) 41% (B) 51% (C) 31% (D) Insufficient information to answer the question (E) 11% (F) 21%

    PaperQA2 Response:
    DiffDomain classifies approximately 30.771% of topologically associated domains (TADs) in the GM12878 blood cell line as reorganized in the K562 cell line. This percentage is significantly higher compared to other methods such as TADCompare, HiCcompare, and HiC-DC+, which identify reorganized TADs at rates of ≤8.256% (hua2024diffdomainenablesidentification pages 4-4). Additionally, DiffDomain demonstrates a higher true positive rate (TPR) and stronger statistical evidence in identifying reorganized TADs compared to alternative approaches (hua2024diffdomainenablesidentification pages 4-5). 

While other excerpts mention that DiffDomain identifies

2025-02-27 20:32:56,158 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


JudgeAgent (to HumanProxy):

correct

--------------------------------------------------------------------------------


2025-02-27 20:32:57,186 - MC_Grader - INFO - Processing question 2/10
2025-02-27 20:32:57,187 - MC_Grader - INFO - Grading question: At least how long do SynNotch-MCF10DCIS cells expr...


...

HumanProxy (to GraderAgent):


    Question:
    How does the presence of m6A differ between x-linked genes and autosomal genes during mouse development?

    Options:
    (A) There is equivalent m6A enrichment between x-linked and autosomal genes during embryonic development (B) m6A enrichment is lower in x-linked genes compared to autosomal genes in the midbrain and hippocampus (C) Autosomes tend to have greater m6A enrichment during embryonic development (D) Insufficient information to answer the question (E) m6A tends to be less abundant within autosomal genes than x-linked genes during embryonic development

    PaperQA2 Response:
    m6A RNA methylation is consistently less enriched on X-linked transcripts compared to autosomal transcripts during mouse development. This trend is observed across various embryonic and somatic cell types, including embryonic stem cells, neural progenitor cells (NPCs), mesenchymal stem cells (MSCs), and liver tissue. However, in specific brain region

2025-02-27 20:33:19,710 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


GraderAgent (to HumanProxy):

B

--------------------------------------------------------------------------------
HumanProxy (to JudgeAgent):


    Graded answer: B
    Correct answer: C
    Unsure option: D

    Is this "correct", "incorrect", or "unsure"? Return ONLY one word.
    

--------------------------------------------------------------------------------


2025-02-27 20:33:20,310 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


JudgeAgent (to HumanProxy):

incorrect

--------------------------------------------------------------------------------

Grading Results:
Total questions: 10
Correct answers: 5 (50.00%)
Incorrect answers: 5 (50.00%)
Unsure answers: 0 (0.00%)
Precision: 50.00%
